In [ ]:
from itertools import product

import numpy as np
from IPython.display import display, clear_output
from matplotlib import pyplot as plt

In [ ]:
char_to_int = {"#": 0, ".": 1, "E": 2, "S": 3, "*": 4}
char_to_img = np.vectorize(char_to_int.get)

In [ ]:
with open("data/day16/input.txt", "r") as file:
    map_init_raw = file.read()

In [ ]:
map_init = np.array([list(x) for x in map_init_raw.split("\n")])

In [ ]:
dir_vec = {
    0: (0, 1),  # E
    1: (1, 0),  # S
    2: (0, -1),  # W
    3: (-1, 0),  # N
}

## Part 1

In [ ]:
node_locs = np.argwhere(map_init != "#")
node_loc_end = np.argwhere(map_init == "E")[0]
node_loc_start = np.argwhere(map_init == "S")[0]

In [ ]:
nodes = {
    (int(x[0]), int(x[1]), y): {"c": np.inf, "n": None, "v": False}
    for x, y in product(node_locs, range(4))
}

In [ ]:
d_max = np.abs(node_loc_end - node_loc_start).sum()
d_loc = {
    tuple(node_loc): np.abs(node_loc_end - node_loc).sum() / d_max
    for node_loc in node_locs
}

In [ ]:
nodes[tuple(node_loc_start) + (0,)]["c"] = 0
nodes[tuple(node_loc_start) + (0,)]["n"] = set()
nodes[tuple(node_loc_start) + (0,)]["n"].add(
    (int(node_loc_start[0]), int(node_loc_start[1]))
)
nodes[tuple(node_loc_start) + (0,)]

In [ ]:
solved_poss_ind = [tuple(np.argwhere(map_init == "E")[0]) + (i,) for i in range(4)]

In [ ]:
fig, ax = plt.subplots()
iter = 0
d_best = 1

while not min([nodes[ind]["c"] for ind in solved_poss_ind]) < np.inf:

    idx_visit = sorted(
        [(idx, vals["c"]) for idx, vals in nodes.items() if not vals["v"]],
        key=lambda x: x[1],
    )[0][0]
    node = nodes[idx_visit]
    forward = dir_vec[idx_visit[2]]

    for i_offset, offset in enumerate(
        [(0, 0, 1, 1000), (0, 0, -1, 1000), forward + (0, 1)]
    ):

        idx_cons = (
            idx_visit[0] + offset[0],
            idx_visit[1] + offset[1],
            (idx_visit[2] + offset[2]) % 4,
        )
        if idx_cons in nodes:
            if node["c"] + offset[3] < nodes[idx_cons]["c"]:
                nodes[idx_cons]["c"] = node["c"] + offset[3]
                nodes[idx_cons]["n"] = node["n"].copy()
                nodes[idx_cons]["n"].add(idx_cons[:2])
            elif node["c"] + offset[3] == nodes[idx_cons]["c"]:
                nodes[idx_cons]["n"].update(node["n"])
                nodes[idx_cons]["n"].add(idx_cons[:2])

            if i_offset == 2:
                d_curr = d_loc[idx_cons[:2]]
                if d_curr < np.floor(d_best * 100) / 100:
                    d_best = d_curr
                    node_paths = nodes[idx_cons]["n"]
                    map = map_init.copy()
                    for i in list(nodes[idx_cons]["n"]):
                        map[*i] = "*"
                    ax.imshow(char_to_img(map))
                    ax.set_title(f"Iteration: {iter} ({1-d_best:.2f})")
                    display(fig)
                    clear_output(wait=True)

    node["v"] = True
    iter += 1

solved_ind = sorted(
    [(ind, nodes[ind]["c"]) for ind in solved_poss_ind], key=lambda x: x[1]
)[0][0]

map = map_init.copy()
for i in list(nodes[solved_ind]["n"]):
    map[*i] = "*"
ax.imshow(char_to_img(map))
ax.set_title(f"Iteration: {iter} ({1-d_best:.2f})")
display(fig)
clear_output(wait=True)

In [ ]:
nodes[solved_ind]["c"]

## Part 2

In [ ]:
len(nodes[solved_ind]["n"])